<a href="https://colab.research.google.com/github/FedericoFortuna/ProgramacionConcurrente/blob/main/TP1_P3_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Comunicación de procesos en Python (Pipe)**

In [84]:
%%writefile copiar_archivo.py
import os
import sys
from multiprocessing import Process, Pipe


def copiar_chunk(pipe, chunk):
    # Enviar el chunk a través de la tubería
    pipe.send_bytes(chunk)


def copiar_archivo(pipe, archivo_a_copiar, chunk_size):
    # Leer el archivo de origen y enviar los chunks a través de la tubería
    with open(archivo_a_copiar, 'rb') as archivo:
        while True:
            chunk = archivo.read(chunk_size)
            if not chunk:
                break
            copiar_chunk(pipe, chunk)

    # Enviar un mensaje de finalización a través de la tubería
    pipe.send_bytes(b'FINALIZADO')

    # Cerrar la tubería
    pipe.close()


# Obtener el path del archivo a copiar del primer parámetro
archivo_a_copiar = sys.argv[1]

# Obtener el tamaño del chunk del segundo parámetro, o usar el valor predeterminado de 512 si no se proporciona
if len(sys.argv) > 2:
    chunk_size = int(sys.argv[2])
else:
    chunk_size = 512

# Crear la tubería
padre_pipe, hijo_pipe = Pipe()

# Crear el proceso hijo
hijo = Process(target=copiar_archivo, args=(hijo_pipe, archivo_a_copiar, chunk_size))
hijo.start()

# Crear el archivo de destino con _copia al final del nombre
nombre_archivo, extension = os.path.splitext(archivo_a_copiar)
archivo_destino = f"{nombre_archivo}_copia{extension}"

# Leer los chunks de la tubería y escribirlos en el archivo de destino
with open(archivo_destino, 'wb') as archivo:
    while True:
        chunk = padre_pipe.recv_bytes(chunk_size)
        if chunk == b'FINALIZADO':
            break
        archivo.write(chunk)

# Esperar a que el proceso hijo termine
hijo.join()

# Imprimir un mensaje de confirmación
print(f"Se ha creado una copia de {archivo_a_copiar} en {archivo_destino}.")


Overwriting copiar_archivo.py


**Creo el archivo a copiar**

In [85]:
!echo 'texto de prueba' > prueba.txt

In [86]:
!cat prueba.txt

texto de prueba


**Ejecuto el programa para hacer la copia**

In [87]:
!python copiar_archivo.py prueba.txt 1>salidaPython 2>/dev/null

**Muestro el archivo copiado**

In [88]:
!cat prueba_copia.txt

texto de prueba


In [89]:
!cat salidaPython

Se ha creado una copia de prueba.txt en prueba_copia.txt.


In [90]:
!ls


copiar_archivo.py  prueba_copia.txt  prueba.txt  salidaPython  sample_data
